In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [9]:
class Action:
    BANK_LEFT = 'L'
    BANK_RIGHT = 'R'
    THRUST = 'T'
    FIRE = 'F'
    
    ACTIONS = [BANK_LEFT, BANK_RIGHT, THRUST, FIRE]

class Game:
    """A single game."""
    def __init__(self, actions, states, result):
        self.actions = actions
        self.states = states
        self.result = result
        
    def to_df(self):
        """Outputs a Pandas DF with one row for each time"""
        rows = []
        for i, (act, state) in enumerate(zip(self.actions, self.states)):
            rows.append({
                't': state['t'],
                't_int': i+1,
                'x1': state['players']['0']['position'][0],
                'y1': state['players']['0']['position'][1],
                'vx1': state['players']['0']['velocity'][0],
                'vy1': state['players']['0']['velocity'][1],
                'a1': state['players']['0']['orientation'],
                'x2': state['players']['1']['position'][0],
                'y2': state['players']['1']['position'][1],
                'vx2': state['players']['1']['velocity'][0],
                'vy2': state['players']['1']['velocity'][1],
                'a2': state['players']['1']['orientation'],
                'bullets': [state['bullets'][k] for k in '01']
            })
        rows = pd.DataFrame(rows).set_index('t_int')
        rows['a1_0'] = rows['a1'] + wedge_size / 2
        rows['a1_1'] = rows['a1'] - wedge_size / 2
        rows['a2_0'] = rows['a2'] + wedge_size / 2
        rows['a2_1'] = rows['a2'] - wedge_size / 2
        return rows

In [10]:
width = 50 * 20
height = 50 * 20
player_r = 15
wedge_size = np.pi / 4
c = ['#4287f5', '#ffa30f']

In [11]:
def parse_player_actions(actions):
    return {act: act in actions for act in Action.ACTIONS}

def read_snoopy(filename):
    with open(filename, 'r') as stream:
        actiondata, statedata, resultdata = stream.read().split('\n\n')
    actions = []
    for line in actiondata.split('\n'):
        actions.append([parse_player_actions(acts) for acts in line.split(' ')])

    states = [json.loads(state) for state in statedata.split('\n')]
    res = int(resultdata.strip())
    return Game(actions, states, res)
    

In [12]:
g = read_snoopy("/home/nicholas/IdeaProjects/snoopy-server/replays/game1.snoopy")
df = g.to_df()
df.head()

,t,x1,y1,vx1,vy1,a1,x2,y2,vx2,vy2,a2,bullets,a1_0,a1_1,a2_0,a2_1
t_int,,,,,,,,,,,,,,,,
1,0.03125,664.750,666.500,-247.750,-23.250,3.250,335.250,333.250,247.750,-7.750,0.125,"[[], []]",3.642699,2.857301,0.517699,-0.267699
2,0.06250,657.125,665.750,-232.875,-37.250,3.375,342.875,332.875,232.875,-22.625,0.250,"[[], []]",3.767699,2.982301,0.642699,-0.142699
3,0.09375,649.875,664.375,-219.500,-50.500,3.500,352.000,332.500,451.125,31.750,0.375,"[[], []]",3.892699,3.107301,0.767699,-0.017699
4,0.12500,643.125,662.750,-207.375,-63.125,3.625,367.500,334.125,618.125,108.375,0.500,"[[], []]",4.017699,3.232301,0.892699,0.107301
5,0.15625,636.750,660.625,-196.375,-75.250,3.750,387.750,338.125,726.000,197.625,0.625,"[[], []]",4.142699,3.357301,1.017699,0.232301


In [13]:
names = ['Player 1', 'Player 2']
def parse_bullets(bullets):
    rows = []
    for p_bullets, name in zip(bullets, names):
        for bullet in p_bullets:
            rows.append({
                'x': bullet['position'][0],
                'y': bullet['position'][1],
                'player': name
            })
    return pd.DataFrame(rows)

In [14]:
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, Button
from bokeh.plotting import figure
from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.models import CategoricalColorMapper
from bokeh.resources import CDN
from bokeh.embed import server_document



output_notebook()

def app(doc):
    
    player_map = CategoricalColorMapper(factors=names, palette=c)

    t = 1
    scale = 1 / 1.5
    source = df.loc[[t], :]
    
    
    def plot(t):
        # Set up plot
        plot = figure(height=round(height * scale), width=round(width * scale), title="Game",
                      tools="crosshair,save",
                      x_range=[0, width], y_range=[0, height])

        players = df.loc[[t], :]
        bullets = parse_bullets(df.loc[t, 'bullets'])
        plot.wedge('x1', 'y1', start_angle='a1_0', end_angle='a1_1', color=c[0], radius=player_r, legend_label='Player 1', source=players)
        plot.wedge('x2', 'y2', start_angle='a2_0', end_angle='a2_1', color=c[1], radius=player_r, legend_label='Player 2', source=players)
        if not bullets.empty:
            plot.circle('x', 'y', color={'field': 'player', 'transform': player_map}, source=bullets)
        return plot


    # Set up widgets
    time = Slider(value=min(df.index), start=min(df.index), end=max(df.index), step=1)

    def update_data(attrname, old, new):
        col.children[0] = plot(time.value)

    time.on_change('value', update_data)
    
    playing = False
    def animate_update():
        global playing
        if 'playing' not in globals():
            pass
        elif playing:
            t = time.value + 1
            if t not in df.index:
                t = 1
            time.value = t

    callback_id = None

    def animate():
        global playing
        if button.label == '► Play':
            playing = True
            button.label = '❚❚ Pause'
        else:
            button.label = '► Play'
            playing = False
            
    callback_id = doc.add_periodic_callback(animate_update, 200)

    button = Button(label='► Play', width=40)
    button.on_click(animate)


    # Set up layouts and add to document
    col = column(plot(1), row(time, button))
    

    doc.add_root(col)
    doc.title = "Game"
    
show(app)

Loading BokehJS ...